## Final Project Submission

Please fill out:
* Student name: Isabella Siele
* Student pace:  full time
* Scheduled project review date/time: 3/10/2025
* Instructor name: Samuel Karu
* Blog post URL:


# Aviation Accident Risk Analysis
## 1.Overview

The company is planning to expand into the aviation industry to diversify its portfolio. The idea is to purchase and operate aircraft for commercial and private use. The project will thus use data to provide insights and guide on decisions to make by cleaning, analysing and visualizations.

## 2.Problem Statement

The company does not have much information on aviation safety or accident patterns. This may expose the business to unnecessary risks like financial losses and safety concerns.Thus the  project aims to determine which aircraft has the lowest risk for the company to start this new business and what factors to consider which influence the occurence of accidents. 

## 3.Objectives

This project will focus on answering three main questions:

-Which aircraft make and categories have the lowest accident and fatality rates?It helps to identify safe aircraft types to prioritize during purchase.

-What outside factors(Weather conditions)play the biggest role  in accidents?

-How does the purpose of flight(commercial)affect accident risks?



## Data Understanding

In [ ]:
# Importing needed libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# loading data
df = pd.read_csv(".\data\Aviation_Data.csv")

In [ ]:
# Top 5 rows in the dataset
df.head()

In [ ]:
# last 5 rows in the dataset
df.tail()

In [ ]:
# Shows Number of rows and columns in dataset
df.shape

In [ ]:
# Info of dataset
df.info()

In [ ]:
# Columns in dataset
df.columns

In [ ]:
# The datatypes in columns
df.dtypes

## Data Preparation

In [ ]:
# checking for missing values
df.isna().sum().sort_values(ascending=False)